# 04. 토치텍스트(TorchText)의 batch_first

이번 챕터에서는 토치텍스트에서 배치퍼스트(batch_first)를 True로 한 경우와 False를 한 경우를 비교해보겠습니다. 이번 챕터는 토치텍스트 튜토리얼 챕터가 아니므로, 토치텍스트를 이전 챕터를 통해 이미 알고 있다고 가정합니다.

## 1. 훈련 데이터와 테스트 데이터로 분리하기

In [1]:
import urllib.request
import pandas as pd

In [ ]:
# 우선 인터넷에서 IMDB 리뷰 데이터를 다운로드 받습니다.

urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")


다운로드한 IMDB 리뷰 데이터를 데이터프레임에 저장합니다.



In [2]:
df = pd.read_csv('../dataset/IMDB/IMDb_Reviews.csv', encoding='latin1')
# 전체 샘플의 개수를 보겠습니다.

In [3]:
print('전체 샘플의 개수 : {}'.format(len(df)))


전체 샘플의 개수 : 50000


전체 샘플의 개수는 50,000개입니다. 25,000개씩 분리하여 훈련 데이터와 테스트 데이터로 분리해보겠습니다.



In [4]:
train_df = df[:25000]
test_df = df[25000:]

In [6]:
train_df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [ ]:
#상위 25,000개의 행은 train_df에 하위 25,000개의 행은 test_df에 저장하였습니다.

train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

***

## 2. 필드 정의하기(torchtext.data)

torchtext.data를 통해 필드(Field)를 정의합니다.


In [7]:
from torchtext import data # torchtext.data import

# 필드 정의
TEXT = data.Field(sequential=True,
                 use_vocab=True,
                 tokenize=str.split,
                 lower=True,
                 batch_first=True, # <== 이 부분을 True로 합니다.
                 fix_length=20)

LABEL = data.Field(sequential=False,
                  use_vocab=False,
                  batch_first=False,
                  is_target=True)

/Users/ohyeji/anaconda3/envs/torch/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


- batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)
    
batch_first 인자를 True로 합니다.

***

## 3. 데이터셋 / 단어 집합 / 데이터로더 만들기


In [15]:
from torchtext.data import TabularDataset
from torchtext.data import Iterator

# TabularDataset은 데이터를 불러오면서 필드에서 정의했던 토큰화 방법으로 토큰화를 수행합니다.
train_data, test_data = TabularDataset.splits(path='../dataset/IMDB/',
                                             train='train_data.csv',
                                             test='test_data.csv',
                                              format='csv',
                                             fields=[('text', TEXT), ('label', LABEL)],
                                             skip_header=True)

# 정의한 필드에 .build_vocab() 도구를 사용하면 단어 집합을 생성합니다.
TEXT.build_vocab(train_data, min_freq=10, max_size=10000) # 10,000개의 단어를 가진 단어 집합 생성

# 배치 크기를 정하고 첫번째 배치를 출력해보겠습니다.
batch_size = 5
train_loader = Iterator(dataset=train_data, batch_size=batch_size)
batch = next(iter(train_loader)) # 첫번째 미니배치

In [12]:
print(batch.text)

tensor([[   3,   20,   42,    3,  718,    5,   46,  232,    5,    0,   35,  170,
            6,   90,    3,  782,   19,    3,  163,  232],
        [ 124,   64,   30,    5,  134, 4422,   50,   24,  361, 2233,    4,   39,
          567,   19,    2, 9075,    6,   66,   48,   14],
        [  44,   10,   25,   64,   71,  561,   31,    0, 5131,   40, 3696,    0,
           11,   54,   28, 4947,   16,   48,   11, 4028],
        [  15,    3, 9647,    9,  260,   10,   20,  155,   84,   11,   14,   19,
          269,  356,   11,   14,   19,    3,    0,   76],
        [ 319,   12,   10,   20,   14,  249,  371,    8,  340,   68,    3,  403,
          212, 1272,   81,    0,    0,  279,   69,  633]])


배치 크기가 5이기 때문에 5개의 샘플이 출력되는 것을 볼 수 있습니다. 각 샘플의 길이는 20의 길이를 가지는데, 이는 앞서 초기에 필드를 정의할 때 fix_length를 20으로 정해주었기 때문입니다. 다시 말해 하나의 미니 배치의 크기는 (배치 크기 × fix_length)입니다.

미니 배치의 크기를 출력해봅시다.

In [13]:
print(batch.text.shape)

torch.Size([5, 20])


***

## 4. 필드 재정의하기(torchtext.data)


In [14]:
# 필드 정의
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=str.split,
                  lower=True,
                  fix_length=20)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

TEXT 필드에서 batch_first = True 인자를 제거하였습니다. 기본값이 False이므로 batch_first의 값이 False가 됩니다.



***
# 5. batch_first = False로 하였을 경우의 텐서 크기

필드를 재정의하였으므로 데이터셋, 단어 집합, 데이터로더를 만드는 과정을 다시 수행해주어야 합니다. __3. 데이터셋 / 단어 집합 / 데이터로더 만들기__의 코드를 재수행했다고 가정합니다.



In [16]:
# 첫번째 미니 배치 출력
print(batch.text)

tensor([[   3,  124,   44,   15,  319],
        [  20,   64,   10,    3,   12],
        [  42,   30,   25, 9647,   10],
        [   3,    5,   64,    9,   20],
        [ 718,  134,   71,  260,   14],
        [   5, 4422,  561,   10,  249],
        [  46,   50,   31,   20,  371],
        [ 232,   24,    0,  155,    8],
        [   5,  361, 5131,   84,  340],
        [   0, 2233,   40,   11,   68],
        [  35,    4, 3696,   14,    3],
        [ 170,   39,    0,   19,  403],
        [   6,  567,   11,  269,  212],
        [  90,   19,   54,  356, 1272],
        [   3,    2,   28,   11,   81],
        [ 782, 9075, 4947,   14,    0],
        [  19,    6,   16,   19,    0],
        [   3,   66,   48,    3,  279],
        [ 163,   48,   11,    0,   69],
        [ 232,   14, 4028,   76,  633]])


하나의 미니 배치의 크기는 (fix_length × 배치 크기)입니다.

In [17]:
print(batch.text.shape)

torch.Size([20, 5])
